# AWS Bedrock RAG Infrastructure Setup

This notebook guides you through creating AWS infrastructure for a Bedrock RAG implementation.
**VPC creation is handled by an external script (`create_vpc_script.py`).**
Subsequent resources (Aurora, S3, Bedrock KB) are created step-by-step in this notebook.

# 1. Imports
Import necessary libraries

In [11]:
import boto3
import json
from botocore.exceptions import ClientError
import time
import zipfile
import os
import io

# 2. Configuration
Define configuration variables for the infrastructure.

In [ ]:
AGENT_NAME = "AppDemoAgentWKB"  # Choose a unique name for your agent
KNOWLEDGE_BASE_ID = "" # e.g., "ABCDEFGHIJ"
KNOWLEDGE_BASE_ARN = "" # e.g., "arn:aws:bedrock:us-east-1:123456789012:knowledge-base/ABCDEFGHIJ"
AGENT_EXECUTION_ROLE_ARN = "" # e.g., "arn:aws:iam::123456789012:role/MyBedrockAgentRole"
FOUNDATION_MODEL_ARN = "arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-lite-v1:0" # Example: Change to your desired model/region

# ** OPTIONAL: Customize these values **
AGENT_DESCRIPTION = "Agent specialized in construction and heavy machines with access to knowledgebase and data in relationaldatabase"
AGENT_INSTRUCTION = """
You are a helpful customer support agent.
Your goal is to answer customer questions accurately based *only* on the information provided in the attached knowledge base.
If the knowledge base does not contain the answer, clearly state that the information is not available in the provided documents.
Do not make up answers or use external knowledge.
Be polite and concise.
"""
IDLE_SESSION_TTL_SECONDS = 1800  # 30 minutes timeout
AWS_REGION = "us-west-2" # Change if your KB/resources are in a different region


In [3]:
try:
    bedrock_agent_client = boto3.client('bedrock-agent', region_name=AWS_REGION)
except Exception as e:
    print(f"Error creating Boto3 client: {e}")
    exit()

In [4]:
knowledge_base_configuration = [
    {
        'knowledgeBaseId': KNOWLEDGE_BASE_ID,
        'description': "Use this knowledge base to find information about product features, setup, and troubleshooting."
    }
]

# 3. Create Bedrock Agent
Creates a Bedrock Agent that connects with a lambda function and a previously created knowledgebase.

In [5]:
FUNCTION_NAME = "ApplicationDemoDBRetrieval" # Choose a unique name for your Lambda function
IAM_ROLE_NAME = "MyLambdaBasicExecutionRole" 

BASIC_EXECUTION_POLICY_ARN = "arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"

# ** OPTIONAL: Customize these values **
FUNCTION_DESCRIPTION = "Lambda function to retrieve database data for Bedrock Agent."
RUNTIME = "python3.12" # Choose a supported Python runtime (e.g., python3.9, python3.10, python3.11, python3.12)
HANDLER = "lambda_function.lambda_handler" # Standard handler for Python: filename.function_name
MEMORY_SIZE = 128 # Memory in MB
TIMEOUT = 30 # Timeout in seconds
AWS_REGION = "us-west-2" # Change to your desired AWS region

In [6]:
assume_role_policy_document = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
})

In [7]:
iam_client = boto3.client('iam', region_name=AWS_REGION)

In [8]:
create_role_response = iam_client.create_role(
        RoleName=IAM_ROLE_NAME,
        AssumeRolePolicyDocument=assume_role_policy_document,
        Description="IAM role assumed by the MyOrderStatusChecker Lambda function for basic execution permissions (CloudWatch Logs)."
        # You can add tags here if needed:
        # Tags=[{'Key': 'Project', 'Value': 'BedrockAgentDemo'}]
    )
role_arn = create_role_response['Role']['Arn']
role_id = create_role_response['Role']['RoleId']

LAMBDA_EXECUTION_ROLE_ARN = role_arn

In [9]:
lambda_code = """
import json
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

def lambda_handler(event, context):
    '''
    Handles the incoming event from the Bedrock Agent Action Group.
    Expects the orderId to be passed via pathParameters.
    '''
    logger.info(f"Received event: {json.dumps(event)}")

    response_code = 200
    response_body = {}

    try:
        # Bedrock Agent invoking a Lambda via API Gateway definition
        # often structures the event like an API Gateway proxy event.
        # Check for pathParameters first.
        if 'pathParameters' in event and event['pathParameters'] and 'orderId' in event['pathParameters']:
             order_id = event['pathParameters']['orderId']
             logger.info(f"Extracted orderId: {order_id} from pathParameters")
        # Fallback: Check if the input payload itself contains the orderId (less common for GET via OpenAPI path)
        elif 'orderId' in event:
             order_id = event['orderId']
             logger.info(f"Extracted orderId: {order_id} from event body")
        else:
            # If Bedrock passes parameters differently (e.g., in requestBody), adjust extraction logic here.
            # The exact structure depends on how Bedrock maps the OpenAPI spec to the Lambda input.
            # Checking the actual event logged by Lambda during testing is crucial.
            logger.warning("Could not find 'orderId' in expected locations (event.pathParameters.orderId or event.orderId)")
            # Attempt to find it in a potential request body if content type was application/json in OpenAPI
            request_body_str = event.get('requestBody', {}).get('content', {}).get('application/json', {}).get('body', '{}')
            try:
                request_body = json.loads(request_body_str)
                if 'orderId' in request_body.get('properties', {}): # Check based on common agent structures
                     order_id_prop = next((prop for prop in request_body['properties'] if prop['name'] == 'orderId'), None)
                     if order_id_prop:
                         order_id = order_id_prop['value']
                         logger.info(f"Extracted orderId: {order_id} from requestBody properties")
                     else:
                         raise ValueError("orderId property not found in requestBody")
                else:
                     raise ValueError("orderId not found in requestBody properties")
            except (json.JSONDecodeError, ValueError, KeyError) as e:
                logger.error(f"Failed to extract orderId from requestBody: {e}")
                raise ValueError("Missing required parameter: orderId")


        # --- Mock Order Status Logic ---
        # Replace this with your actual logic to check order status (e.g., database query)
        mock_database = {
            "12345": {"status": "Shipped", "estimatedDelivery": "2025-05-10"},
            "67890": {"status": "Processing", "estimatedDelivery": "2025-05-15"},
            "ABCDE": {"status": "Delivered", "estimatedDelivery": "2025-05-01"}
        }

        if order_id in mock_database:
            order_details = mock_database[order_id]
            response_body = {
                "orderId": order_id,
                "status": order_details["status"],
                "estimatedDelivery": order_details["estimatedDelivery"]
            }
            logger.info(f"Found order details: {response_body}")
        else:
            logger.warning(f"Order ID '{order_id}' not found.")
            response_code = 404
            response_body = {"message": f"Order with ID '{order_id}' not found."}
        # --- End Mock Logic ---

    except ValueError as ve: # Specific error for missing parameter
        logger.error(f"Value Error: {ve}")
        response_code = 400 # Bad Request
        response_body = {"message": str(ve)}
    except Exception as e:
        logger.error(f"An unexpected error occurred: {e}", exc_info=True)
        response_code = 500 # Internal Server Error
        response_body = {"message": "Internal server error while processing the order status."}

    # Format the response payload expected by the Bedrock Agent/API Gateway integration
    # The agent expects a JSON response matching the OpenAPI schema's '200' response definition.
    return {
        'statusCode': response_code,
        'headers': {
            'Content-Type': 'application/json'
        },
        'body': json.dumps(response_body)
        # Bedrock Agent might simplify this; check agent trace/logs if needed.
        # Sometimes it just needs the core JSON body if not using API Gateway directly.
        # For direct Lambda invocation via action group, the structure might be simpler:
        # return response_body # If statusCode/headers aren't needed by Bedrock directly
    }

"""

In [12]:
zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Write the lambda code to a file named lambda_function.py within the zip
    # The filename must match the first part of the HANDLER variable
    zipf.writestr('lambda_function.py', lambda_code)

zip_buffer.seek(0) # Rewind the buffer to the beginning
deployment_package = zip_buffer.read()

In [13]:
lambda_client = boto3.client('lambda', region_name=AWS_REGION)

response = lambda_client.create_function(
        FunctionName=FUNCTION_NAME,
        Runtime=RUNTIME,
        Role=LAMBDA_EXECUTION_ROLE_ARN,
        Handler=HANDLER,
        Code={'ZipFile': deployment_package},
        Description=FUNCTION_DESCRIPTION,
        Timeout=TIMEOUT,
        MemorySize=MEMORY_SIZE,
        Publish=True,  # Automatically publish the first version
        # Add Environment variables, VPC config, etc. here if needed
        # Environment={ 'Variables': { 'MY_VAR': 'my_value' } },
        # VpcConfig={ 'SubnetIds': ['subnet-xxxxx'], 'SecurityGroupIds': ['sg-yyyyy'] }
    )



In [14]:
function_arn = response.get('FunctionArn')
print("\n--- Lambda Function Creation Successful ---")
print(f"Function Name: {response.get('FunctionName')}")
print(f"Function ARN: {function_arn}")
print(f"Runtime: {response.get('Runtime')}")
print(f"Handler: {response.get('Handler')}")
print(f"Role ARN: {response.get('Role')}")
print(f"Version: {response.get('Version')}")
print("-----------------------------------------")
print(f"\nUse this ARN for the LAMBDA_FUNCTION_ARN in your Bedrock Agent Action Group script:")
print(function_arn)


--- Lambda Function Creation Successful ---
Function Name: ApplicationDemoDBRetrieval
Function ARN: arn:aws:lambda:us-west-2:058264544288:function:ApplicationDemoDBRetrieval
Runtime: python3.12
Handler: lambda_function.lambda_handler
Role ARN: arn:aws:iam::058264544288:role/MyLambdaBasicExecutionRole
Version: 1
-----------------------------------------

Use this ARN for the LAMBDA_FUNCTION_ARN in your Bedrock Agent Action Group script:
arn:aws:lambda:us-west-2:058264544288:function:ApplicationDemoDBRetrieval


In [15]:
LAMBDA_FUNCTION_ARN = function_arn

In [16]:
OPENAPI_SCHEMA = {
    "openapi": "3.0.0",
    "info": {
        "title": "Wikipedia Finder API",
        "version": "1.0.0"
    },
    "paths": {
        "/wikipedia": {
            "post": {
                "summary": "Get Wikipedia URL for celebrity",
                "parameters": [
                    {
                        "name": "celebrity_name",
                        "in": "query",
                        "required": True,
                        "schema": {"type": "string"}
                    }
                ],
                "responses": {
                    "200": {
                        "description": "Success response",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "type": "object",
                                    "properties": {
                                        "url": {"type": "string"}
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
    }
}

In [17]:
ACTION_GROUP_CONFIG = [
    {
        'actionGroupName': 'WikipediaAPI',
        'description': 'Fetches Wikipedia URLs using OpenAPI schema',
        'actionGroupExecutor': {
            'lambda': LAMBDA_FUNCTION_ARN
        },
        'apiSchema': {
            'payload': json.dumps(OPENAPI_SCHEMA)  # Use S3 reference for large schemas
        }
    }
]

In [20]:
response = bedrock_agent_client.create_agent(
    agentName=AGENT_NAME,
    agentResourceRoleArn=AGENT_EXECUTION_ROLE_ARN,
    foundationModel=FOUNDATION_MODEL_ARN,
    instruction=AGENT_INSTRUCTION,
    description=AGENT_DESCRIPTION,
    idleSessionTTLInSeconds=IDLE_SESSION_TTL_SECONDS
)

agent_details = response.get('agent', {})
agent_id = agent_details.get('agentId')
agent_arn = agent_details.get('agentArn')
agent_status = agent_details.get('agentStatus')

print("\n--- Agent Creation Initiated ---")
print(f"Agent Name: {agent_details.get('agentName')}")
print(f"Agent ID: {agent_id}")
print(f"Agent ARN: {agent_arn}")
print(f"Initial Status: {agent_status}")
print("------------------------------")
print("\nAgent creation can take a few minutes.")
print("Check the AWS Bedrock console for the current status.")


--- Agent Creation Initiated ---
Agent Name: AppDemoAgentWKB
Agent ID: ZEFGZNWHS0
Agent ARN: arn:aws:bedrock:us-west-2:058264544288:agent/ZEFGZNWHS0
Initial Status: CREATING
------------------------------

Agent creation can take a few minutes.
Check the AWS Bedrock console for the current status.


In [21]:
while agent_status not in ['PREPARED', 'FAILED']:
    print(f"Current agent status: {agent_status}. Waiting...")
    time.sleep(30) # Check every 30 seconds
    get_agent_response = bedrock_agent_client.get_agent(agentId=agent_id)
    agent_status = get_agent_response.get('agent', {}).get('agentStatus')
    
    if agent_status == 'PREPARED':
        print(f"\nAgent '{agent_id}' is now PREPARED and ready to use.")
    elif agent_status == 'FAILED':
        print(f"\nAgent '{agent_id}' FAILED preparation. Check console for details.")



Current agent status: CREATING. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...
Current agent status: NOT_PREPARED. Waiting...

Agent 'ZEFGZNWHS0' is now PREPARED and ready to use.


In [22]:
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='string',
    knowledgeBaseId=KNOWLEDGE_BASE_ID,
    knowledgeBaseState='ENABLED'
)